In [5]:
import requests
from pprint import pprint

session = requests.session()

In [6]:
from html import unescape
from bs4 import BeautifulSoup
import re
import random

In [7]:
response = session.get('https://www.kinopoisk.ru/reviews/type/comment/status/good/#list')

In [8]:
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

Открываем страницу с 50 положительными отзывами

In [9]:
url = f'https://www.kinopoisk.ru/reviews/type/comment/status/good/period/month/perpage/50/#list'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

In [10]:
soup = BeautifulSoup(page, 'html.parser')

Находим тег, внутри которого лежат тексты рецензий

In [11]:
reviews = soup.find_all('span', {'class': '_reachbanner_'})

In [12]:
reviews_str = str(reviews)

Чистим текст сначала от лишних знаков

In [13]:
cleaned_str = re.sub('\xa0', ' ', reviews_str)

Оставляем только кириллицу

In [14]:
reg = re.compile('[^а-яёА-ЯЁ ]')
pos_reviews = reg.sub(' ', cleaned_str)

Все то же самое делаем для негативных рецензий и тестовых

In [15]:
url = f'https://www.kinopoisk.ru/reviews/type/comment/status/bad/#list'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

In [16]:
soup = BeautifulSoup(page, 'html.parser')

In [17]:
reviews_neg = soup.find_all('span', {'class': '_reachbanner_'})

In [18]:
reviews_str_neg = str(reviews_neg)

In [19]:
cleaned_str_neg = re.sub('\xa0', ' ', reviews_str_neg)

In [20]:
reg = re.compile('[^а-яёА-ЯЁ ]')
neg_reviews = reg.sub(' ', cleaned_str_neg)

In [21]:
url = f'https://www.kinopoisk.ru/reviews/type/comment/period/month/perpage/25/#list'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

In [22]:
soup = BeautifulSoup(page, 'html.parser')

In [23]:
reviews_test = soup.find_all('span', {'class': '_reachbanner_'})

In [24]:
reviews_for_test = str(reviews_test)
r = reviews_for_test.split('<span class="_reachbanner_" itemprop="reviewBody">')

Здесь уже нужно, чтобы каждый текст был отдельно, поэтому разделяем их и кладем 10 рецензий в список

In [25]:
test = []
for rew in r:
    cleaned_str_test = re.sub('\xa0', ' ', rew)
    #print(cleaned_str_test)
    reg = re.compile('[^а-яёА-ЯЁ ]')
    test_reviews = reg.sub(' ', cleaned_str_test)
    #print(test_reviews)
    #while len(test) < 10:
    test.append(test_reviews)
test_list = []
test_list.append(test[1:11])
fin_list = test_list[0]
#print(fin_list[0])

Создаем вектор правильных ответов. 1 - положительный отзыв, 0 - отрицательный

In [85]:
label = [1, 1, 0, 1, 0, 0, 1, 0, 1, 1]

Датафрейм просто так, чтобы проверить, все ли в порядке 

In [86]:
import pandas as pd

In [87]:
df = pd.DataFrame(list(zip(fin_list, label)), columns =['review', 'label']) 

In [88]:
df

,review,label
0,Порой наступает такой момент когда в жизни на...,1
1,В российский прокат вышла драматическая картин...,1
2,Действие фильма происходит во французской тюрь...,0
3,В последнее время я поймал себя на мысли касат...,1
4,Чем можно убить сценарий детективного боевика ...,0
5,Южнокорейский Поезд в Пусан вышедший в ...,0
6,Патрик Суэйзи прекрасен во всех своих работах ...,1
7,Первый сезон я бы сказала был очень своеобра...,0
8,Решила посмотреть после того как увидела в гл...,1
9,Кровавый спорт безусловно можн...,1


In [30]:
import nltk
from nltk.tokenize import word_tokenize

In [31]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Функция для приведения к нижнему регистру, токенизации, лемматизации

In [32]:
def text_prep(text):
    text = text.lower()
    tokenized = word_tokenize(text)
    lemmas = []
    for word in tokenized:
        ana = morph.parse(word)
        first = ana[0]
        lemma = first.normal_form
        lemmas.append(lemma)
    return(lemmas)

Делаем разные множества

In [33]:
positive_set = set(text_prep(pos_reviews))

In [34]:
negative_set = set(text_prep(neg_reviews))

In [35]:
p = positive_set.difference(negative_set) #уникальные элементы только в позитивных отзывах

In [36]:
n = negative_set.difference(positive_set) #уникальные элементы только в негативных отзывах

In [37]:
positive_list = text_prep(pos_reviews)

In [38]:
negative_list = text_prep(neg_reviews)

Ищем редкие слова и удаляем их из множеств

In [39]:
import collections
from collections import Counter

In [40]:
c = collections.Counter(positive_list)
c_list = c.items()

In [41]:
rare_words_pos = []
for el in c_list:
    if el[1] == 1 or el[1] == 2:
        rare_words_pos.append(el[0])

In [42]:
new_pos = []
for word in positive_list:
    if word not in rare_words_pos:
        new_pos.append(word)

In [43]:
c_neg = collections.Counter(negative_list)
c_list_neg = c_neg.items()

In [44]:
rare_words_neg = []
for el in c_list:
    if el[1] == 1 or el[1] == 2:
        rare_words_neg.append(el[0])

In [45]:
new_neg = []
for word in negative_list:
    if word not in rare_words_neg:
        new_neg.append(word)

In [46]:
new_pos_set = set(new_pos)

In [47]:
new_neg_set = set(new_neg)

Делаем сеты из списков, из которых с помощью Counter убраны слова, встречающиеся 1-2 раза

In [48]:
unique_positive = new_pos_set.difference(new_neg_set)

In [49]:
unique_negative = new_neg_set.difference(new_pos_set)

Функция, определяющая положительная рецензия или отрицательная

In [100]:
def super_definer(text):
    y = []
    positive_words = 0
    negative_words = 0
    for text in fin_list:
        text = text_prep(text)
        for word in text:
            if word in unique_positive:
                positive_words += 1
            if word in unique_negative:
                negative_words += 1
        if positive_words >= negative_words:
            y.append(1)
        if negative_words > positive_words:
            y.append(0)
    return(y)

In [101]:
for text in fin_list:
    y = super_definer(text)
print(y)

[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]


In [90]:
from sklearn.metrics import accuracy_score

In [102]:
accuracy_score(y, label)

0.5

# Улучшаем программу

In [55]:
from pymystem3 import Mystem

In [56]:
my = Mystem()

In [108]:
ana = my.analyze(pos_reviews)

In [62]:
mystem_tags = []
for word in ana:
    if 'analysis' in word:
        if len(word['analysis']) > 0:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            llist = []
            llist.append(word['text'])
            llist.append(pos)
            mystem_tags.append(llist)
        #print(llist)
print(mystem_tags)

[['Порой', 'S'], ['наступает', 'V'], ['такой', 'APRO'], ['момент', 'S'], ['когда', 'CONJ'], ['в', 'PR'], ['жизни', 'S'], ['наступает', 'V'], ['черная', 'A'], ['полоса', 'S'], ['и', 'CONJ'], ['сложности', 'S'], ['одолевают', 'V'], ['верх', 'S'], ['превалируя', 'V'], ['над', 'PR'], ['яркими', 'A'], ['эмоциями', 'S'], ['и', 'CONJ'], ['тогда', 'ADVPRO'], ['в', 'PR'], ['тот', 'APRO'], ['момент', 'S'], ['когда', 'ADVPRO'], ['тучи', 'S'], ['застилают', 'V'], ['собой', 'SPRO'], ['солнце', 'S'], ['на', 'PR'], ['помощь', 'S'], ['приходит', 'V'], ['Миссис', 'S'], ['Даутфайр', 'S'], ['Вероятно', 'ADV'], ['это', 'PART'], ['тот', 'APRO'], ['фильм', 'S'], ['который', 'APRO'], ['каждому', 'APRO'], ['стоит', 'V'], ['увидеть', 'V'], ['неважно', 'ADV'], ['будь', 'V'], ['то', 'CONJ'], ['в', 'PR'], ['пору', 'S'], ['глубокого', 'A'], ['отчаяния', 'S'], ['или', 'CONJ'], ['безграничной', 'A'], ['радости', 'S'], ['Это', 'PART'], ['история', 'S'], ['которая', 'APRO'], ['несомненно', 'ADV'], ['заслуживает', 'V']

Берем не + глагол, сущ + прил, наречие + глагол.
Кажется, не + глагол поможет нам выбрать словосочетания типа фильм НЕ ПОНРАВИЛСЯ и поможет лучше определить окраску текста.
Сущ + прил: ожидается выбор словосочетаний типа хороший фильм, плохой фильм, положительные эмоции, отрицательные эмоции.
Наречие + глагол: ожидаются словосочетания ужасно снят, плохо играл (об актере).

In [71]:
dict_bigrams = []
for i in range(len(mystem_tags)-1):
    first_lemma = mystem_tags[i][0]
    first_tag = mystem_tags[i][1]
    second_lemma = mystem_tags[i+1][0]
    second_tag = mystem_tags[i+1][1]
    bigram = []
    if first_lemma == 'не' and second_tag == 'V':
        bigram.append(first_lemma)
        bigram.append(second_lemma)
        dict_bigrams.append(' '.join(bigram))
    if first_tag == 'S' and second_tag == 'A':
        bigram.append(first_lemma)
        bigram.append(second_lemma)
        dict_bigrams.append(' '.join(bigram))
    if first_tag == 'ADV' and second_tag == 'V':
        bigram.append(first_lemma)
        bigram.append(second_lemma)
        dict_bigrams.append(' '.join(bigram))
    #print(first_lemma, first_tag, second_lemma, second_tag)

In [72]:
dict_bigrams

['неважно будь',
 'пору глубокого',
 'несомненно заслуживает',
 'Даутфайр добрая',
 'отважно решившейся',
 'воспитании собственных',
 'не стал',
 'раскрытию наилучших',
 'восприятие обыденных',
 'звание доброго',
 'заряд положительной',
 'платье настоящее',
 'множество интересных',
 'значимость семейных',
 'невозможно перестать',
 'В российский',
 'Фарго Американская',
 'Легион Ходячие',
 'сейчас решился',
 'разнообразиях жанровых',
 'сюжету перспективный',
 'результате глупой',
 'не убедившись',
 'актеры авторитетнее',
 'не стремились',
 'формат прошлых',
 'образ главного',
 'принятия неизбежного',
 'игрой главного',
 'прекрасно взаимодействует',
 'действительно радует',
 'точно попадают',
 'немного смущает',
 'судьбы оригинальное',
 'не имеет',
 'тяжко стало',
 'большинства голливудских',
 'кинематографом Южной',
 'уже успел',
 'не выпускал',
 'сомнения грамотного',
 'однозначно впечатлили',
 'однозначно хватает',
 'не успел',
 'уже успел',
 'серия жестоких',
 'Тела молодых',
 'Сотру

То же самое делаем для негативных рецензий

In [73]:
ana_neg = my.analyze(neg_reviews)

In [74]:
mystem_tags_neg = []
for word in ana:
    if 'analysis' in word:
        if len(word['analysis']) > 0:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            llist = []
            llist.append(word['text'])
            llist.append(pos)
            mystem_tags_neg.append(llist)
        #print(llist)
print(mystem_tags_neg)

[['Порой', 'S'], ['наступает', 'V'], ['такой', 'APRO'], ['момент', 'S'], ['когда', 'CONJ'], ['в', 'PR'], ['жизни', 'S'], ['наступает', 'V'], ['черная', 'A'], ['полоса', 'S'], ['и', 'CONJ'], ['сложности', 'S'], ['одолевают', 'V'], ['верх', 'S'], ['превалируя', 'V'], ['над', 'PR'], ['яркими', 'A'], ['эмоциями', 'S'], ['и', 'CONJ'], ['тогда', 'ADVPRO'], ['в', 'PR'], ['тот', 'APRO'], ['момент', 'S'], ['когда', 'ADVPRO'], ['тучи', 'S'], ['застилают', 'V'], ['собой', 'SPRO'], ['солнце', 'S'], ['на', 'PR'], ['помощь', 'S'], ['приходит', 'V'], ['Миссис', 'S'], ['Даутфайр', 'S'], ['Вероятно', 'ADV'], ['это', 'PART'], ['тот', 'APRO'], ['фильм', 'S'], ['который', 'APRO'], ['каждому', 'APRO'], ['стоит', 'V'], ['увидеть', 'V'], ['неважно', 'ADV'], ['будь', 'V'], ['то', 'CONJ'], ['в', 'PR'], ['пору', 'S'], ['глубокого', 'A'], ['отчаяния', 'S'], ['или', 'CONJ'], ['безграничной', 'A'], ['радости', 'S'], ['Это', 'PART'], ['история', 'S'], ['которая', 'APRO'], ['несомненно', 'ADV'], ['заслуживает', 'V']

In [75]:
dict_bigrams_neg = []
for i in range(len(mystem_tags_neg)-1):
    first_lemma = mystem_tags_neg[i][0]
    first_tag = mystem_tags_neg[i][1]
    second_lemma = mystem_tags_neg[i+1][0]
    second_tag = mystem_tags_neg[i+1][1]
    bigram = []
    if first_lemma == 'не' and second_tag == 'V':
        bigram.append(first_lemma)
        bigram.append(second_lemma)
        dict_bigrams_neg.append(' '.join(bigram))
    if first_tag == 'S' and second_tag == 'A':
        bigram.append(first_lemma)
        bigram.append(second_lemma)
        dict_bigrams.append(' '.join(bigram))
    if first_tag == 'ADV' and second_tag == 'V':
        bigram.append(first_lemma)
        bigram.append(second_lemma)
        dict_bigrams_neg.append(' '.join(bigram))
    #print(first_lemma, first_tag, second_lemma, second_tag)

In [76]:
dict_bigrams_neg

['неважно будь',
 'несомненно заслуживает',
 'отважно решившейся',
 'не стал',
 'невозможно перестать',
 'сейчас решился',
 'не убедившись',
 'не стремились',
 'прекрасно взаимодействует',
 'действительно радует',
 'точно попадают',
 'немного смущает',
 'не имеет',
 'тяжко стало',
 'уже успел',
 'не выпускал',
 'однозначно впечатлили',
 'однозначно хватает',
 'не успел',
 'уже успел',
 'более подготовленного',
 'Необходимо отработать',
 'не остановятся',
 'чётко стал',
 'постоянно генерируя',
 'нередко сменяются',
 'необычно разбавляет',
 'не дают',
 'жестоко убивает',
 'не заметить',
 'ещё продолжу',
 'сначала было',
 'постепенно затягивает',
 'не подделываясь',
 'кубарем выкатывается',
 'серьезно играть',
 'точно стоит',
 'не опустились',
 'не запретили',
 'круто изображено',
 'можно назвать',
 'отчасти заимствует',
 'не уступает',
 'органично уничтожавший',
 'уже успел',
 'вообще вышел',
 'не понравился',
 'ещё могу',
 'не смотрелся',
 'можно сказать',
 'не раскрыт',
 'не двигалась'

Теперь добавим словосочетания к отдельным словам из наших тональных словарей

In [77]:
pos_dict_with_bigrams = []
pos_dict_with_bigrams.append(new_pos)
pos_dict_with_bigrams.append(dict_bigrams)

In [80]:
pos_dict_with_bigrams = pos_dict_with_bigrams[0]

In [95]:
pos_dict_with_bigrams = new_pos + dict_bigrams

In [82]:
neg_dict_with_bigrams = []
neg_dict_with_bigrams.append(new_neg)
neg_dict_with_bigrams.append(dict_bigrams_neg)

In [83]:
neg_dict_with_bigrams = neg_dict_with_bigrams[0]

In [96]:
neg_dict_with_bigrams = new_neg + dict_bigrams_neg

Новая функция для определения тональности

In [111]:
def super_definer_upgrade(text):
    y = []
    positive_bigrams = 0
    negative_bigrams = 0
    for text in fin_list:
        #text = text_prep(text)
        for word in text:
            tags = []
            for word in ana:
                if 'analysis' in word:
                    if len(word['analysis']) > 0:
                        gr = word['analysis'][0]['gr']
                        pos = gr.split('=')[0].split(',')[0]
                        llist = []
                        llist.append(word['text'])
                        llist.append(pos)
                        tags.append(llist)
        dict_bigrams_neg = []
        for i in range(len(tags)-1):
            first_lemma = tags[i][0]
            first_tag = tags[i][1]
            second_lemma = tags[i+1][0]
            second_tag = tags[i+1][1]
            bigram = [first_lemma, second_lemma]
        #print(llist)
            if bigram in pos_dict_with_bigrams:
                positive_bigrams += 1
            if bigram in neg_dict_with_bigrams:
                negative_bigrams += 1
        if positive_bigrams >= negative_bigrams:
            y.append(1)
        if negative_bigrams > positive_bigrams:
            y.append(0)
    return(y)

In [ ]:
for text in fin_list:
    y_test = super_definer_upgrade(text)
print(y_test)

In [105]:
accuracy_score(y_test, label)

0.5